# Analisi sul data set Amazon E-commerce Products & Reviews
Il <a href="https://www.kaggle.com/datasets/lazylad99/amazon-e-commerce-product-and-review-dataset">dataset **Amazon E-commerce Products & Reviews**</a> su Kaggle, contiene metadati dettagliati dei prodotti Amazon, recensioni dei clienti e punteggi di sentiment. Questo insieme di dati è particolarmente indicato per analizzare il mercato e i comportamenti d'acquisto.

In [ ]:
#import library
#data analisys
import pandas as pd 
import numpy as np 
import math
#data visualitation
import seaborn as sns
import matplotlib.pyplot as plt 
from matplotlib.dates import DateFormatter, AutoDateLocator
print(pd.__version__, np.__version__, sns.__version__)


## Comprensione dei Dati (EDA)

In [ ]:
#import dataset or dataframe
dfamazon = pd.read_csv('products_with_reviews_clean.csv', low_memory=False)

#leggo le prime 4 righe del dataframe e le visualizzo nello schermo
display(dfamazon.head(4))

### Shape e Struttura base
<i>Obiettivo: capire volume e tipi dati. Molti object da pulire (prezzi, rating).</i>

Il dataframe in analisi ha 6355 righe e 35 colonne.

Le colonne del dataframe nel dettaglio:
- **s_no**: è un numero intero che contiene gli id di prodotti
- **about_item**: è un oggetto e contiene la descrizione breve del prodotto

In [ ]:
#mostrare il numero di righe e il numero di colonne
display('Numero di righe e colonne', dfamazon.shape)
#mostro tutti nomi delle colonne del dataframe
display('Nome delle colonne',dfamazon.columns)
#mostro le distribuzione del tipo valore
display(dfamazon.dtypes.value_counts())

### Statistiche Descrittive
<i>Obiettivo: capire le colonne e gli attributi presenti nel DataFrame.</i>

Dall'analisi descrittiva posso dedurre che abbiamo 3 parametri a cui mancano la stessa quantità di dati pari a 28, nello specifico: helpful_vote_count, review_position e sentiment_score. Invece nella colonna rating la mancanza dei dati è leggermente più elevata, ovvero 35 valori mancanti.

L'analisi del prodotto in generale mostra che questo ha un prezzo medio di 33,70.

Infine le recensioni dei prodotti sono più che positive perchè il loro rating medio con range compreso tra 1-5 è 4.533228.

In [ ]:
#mostrare i nomi delle colonne e i tipi di dati all'interno e se questi dati hanno dei dati mancati
display(dfamazon.info())
#genera un riepilogo statistico automatico delle colonne numeriche - Restituisce una tabella con metriche chiave: count (numero valori non-null), mean (media), std (deviazione standard), min (minimo), 25% (1° quartile), 50% (mediana), 75% (3° quartile), max (massimo), evidenziando distribuzioni, outlier e tendenze
display(dfamazon.describe())
#genera statistiche descrittive per tutte le colonne del DataFrame, inclusi tipi numerici, categorici (object/string), booleani e datetime, anziché solo numeriche. Per colonne numeriche mostra count, mean, std, min, quartili (25%, 50%/mediana, 75%), max; per categoriche aggiunge unique (valori unici), top (valore più frequente), freq (frequenza top). Esclude NaN automaticamente.
display(dfamazon.describe(include='all'))

### Valori mancanti (Missing Values) e Duplicati

#### Dati Mancanti isnull()

In [ ]:
#creo una variabile che contiene tutti i dati per colonna null
valuesmissing = dfamazon.isnull().sum()
#controllo tutti i dati null per colonna e seleziono solo quelli che hanno un valore superiore a 0 e li ordino in modo decrescente
valuesmissing[valuesmissing>0].sort_values(ascending=False)

In [ ]:
#controllo tutti i dati null per colonna e seleziono solo quelli che hanno un valore superiore a 0 e li ordino in modo decrescente
#dividendola per la lunghezza delle righe del dataframe e moltiplicandola per 100 
(valuesmissing[valuesmissing>0].sort_values(ascending=False) / len(dfamazon) * 100).round(3)
#round -> funzione che arrotonda all'intero più vicino

In [ ]:
#CREO DATAFRAME pd.DataFrame
dfamazon_missing = pd.DataFrame(
    #creazione delle colonne NomeColonna : ValoridaAssegnare
    {
        'Colonna': dfamazon.columns,
        'N_Dati_Mancanti':(dfamazon.isnull().sum()).round(2),
        '%_Dati_Mancanti': (dfamazon.isnull().sum()/len(dfamazon)*100).round(2),
    }
    #query per pulire il nuovo DataFrame e ordinare (sort_values) i valori
).query('N_Dati_Mancanti>0').sort_values('N_Dati_Mancanti',ascending=False)

display(dfamazon_missing)

#### Dati Duplicati (duplicated())

In [ ]:
#Valori duplicati per tutto il dataFrame
dfamazon.duplicated().sum()
#Valori duplicati per le colonne asin e review_id del DataFrame
dfamazon[dfamazon.duplicated(subset=['asin','review_id'], keep='first')]

#### Cancellazione dati Mancanti e Duplicati

In [ ]:
#creo copia del dataframe
dfamazoncopy = dfamazon.copy()
dfamazoncopy.head(3)

In [ ]:
dfamazoncopy.columns

In [ ]:
#creo una lista con le colonne spazzatura ovvero le colonne con dati mancanti >60%
cols_trash = dfamazoncopy.columns[dfamazoncopy.isnull().mean()>0.6].tolist()
cols_trash

In [ ]:
#elimino dal dataframe copia la lista delle colonne spazzatura ovvero le colonne con dati mancanti >60%
dfamazoncopy = dfamazoncopy.drop(columns=cols_trash)
dfamazoncopy.shape

#### Controllo il dataFrame dopo la pulizia

In [ ]:
#CREO DATAFRAME pd.DataFrame
dfamazon_missing = pd.DataFrame(
    #creazione delle colonne NomeColonna : ValoridaAssegnare
    {
        'Colonna': dfamazoncopy.columns,
        'N_Dati_Mancanti':(dfamazoncopy.isnull().sum()).round(2),
        '%_Dati_Mancanti': (dfamazoncopy.isnull().sum()/len(dfamazon)*100).round(2),
    }
    #query per pulire il nuovo DataFrame e ordinare (sort_values) i valori
).query('N_Dati_Mancanti>0').sort_values('N_Dati_Mancanti',ascending=False)

display(dfamazon_missing)

### Creazione nuove colonne utili per l'analisi

#### Creo le colonne categoria

In [ ]:
#creo nuova colonna dove inserisco una parte del breadcrumbs
dfamazoncopy['category1'] = dfamazoncopy['breadcrumbs'].str.split(' › ').str[0].fillna('')
dfamazoncopy['category2'] = dfamazoncopy['breadcrumbs'].str.split(' › ').str[1].fillna('')
dfamazoncopy['category3'] = dfamazoncopy['breadcrumbs'].str.split(' › ').str[2].fillna('')
dfamazoncopy['category4'] = dfamazoncopy['breadcrumbs'].str.split(' › ').str[3].fillna('')
dfamazoncopy['category5'] = dfamazoncopy['breadcrumbs'].str.split(' › ').str[4].fillna('')
dfamazoncopy['category6'] = dfamazoncopy['breadcrumbs'].str.split(' › ').str[5].fillna('')
#cancello la colonna breadcrumbs
dfamazoncopy.drop(['breadcrumbs'],axis=1, inplace=True)
dfamazoncopy.head(10)
dfamazoncopy['verified_purchase'].dtype

#### Convertire valori in colonna

In [ ]:
dfamazoncopy['verified_purchase'] = dfamazoncopy['verified_purchase'].map({'True': True, 'False': False, 'true': True, 'false': False, 
        '1': True, '0': False, 1: True, 0: False, True: True, False: False}).fillna(False).astype(bool)
#controllo tipo valore in colonna
dfamazoncopy['verified_purchase'].dtype
display(dfamazoncopy['verified_purchase'].value_counts())

In [ ]:
display(dfamazoncopy.describe(include='all'))

#### Converto dati colonna availability
- True le frasi:
    - In Stock
    - In stock
    - Only 1 left in stock - order soon.
    - Available to ship in 1-2 days
    - This item will be released on May 20, 2025.
- False le frasi:
    - Currently unavailable.
    - Temporarily out of stock.

In [ ]:
mapping_availability = {
    'in stock': True,
    'only 1 left in stock - order soon.': True,
    'only 3 left in stock - order soon.': True,
    'currently unavailable.': False,
    'only 2 left in stock - order soon.': True,
    'only 5 left in stock - order soon.': True,
    'available to ship in 1-2 days': True,
    'this item will be released on may 20, 2025.': True,
    'only 4 left in stock - order soon.': True,
    'temporarily out of stock.': False,
}

def map_availability(text):
    text = str(text).lower().strip()
    # default False se la stringa non è nelle chiavi
    return mapping_availability.get(text, False)

dfamazoncopy['availability_b'] = (
    dfamazoncopy['availability']
      .apply(map_availability)
      .astype(bool)
)

print(dfamazoncopy['availability_b'].value_counts())
#cancello la colonna availability
dfamazoncopy.drop(['availability'],axis=1, inplace=True)
dfamazoncopy.head(5)

#### Controllo colonna rating_count

In [ ]:
dfamazoncopy['rating_count_n'] = (
    dfamazoncopy['rating_count']
    .astype(str)
    .str.replace('ratings','', regex=False) # '8,110 ratings' -> '8,110 '
    .str.strip() #'8,110 ' -> '8,110'
    .str.replace(',','',regex=False) #'8,110' -> '8110'
)
#TRASFORMA STRINGA IN NUMERO
dfamazoncopy['rating_count_n'] = pd.to_numeric(
    dfamazoncopy['rating_count_n'],
    errors='coerce' # trasforma tutti i valori che non possono essere convertiti in numero e i vuoti-> NaN   
).astype('Int64') # intero con supporto a NaN

dfamazoncopy.drop('rating_count',axis=1,inplace=True)

print(dfamazoncopy['rating_count_n'].value_counts())

#### Colonna rating_stars

In [ ]:
dfamazoncopy['rating_stars'].value_counts()
dfamazoncopy['rating_stars_n'] = (
    dfamazoncopy['rating_stars']
    .astype(str)
    .str.replace('out of 5 stars','', regex=False) # '4.5 out of 5 stars' -> '4.5  '
    .str.strip() #'4.5  ' -> '4.5 '
)
#TRASFORMA STRINGA IN NUMERO
dfamazoncopy['rating_stars_n'] = pd.to_numeric(
    dfamazoncopy['rating_stars_n'],
    errors='coerce' # trasforma tutti i valori che non possono essere convertiti in numero e i vuoti-> NaN   
).astype('Float64') # float con supporto a NaN
dfamazoncopy.drop('rating_stars',axis=1,inplace=True)
dfamazoncopy['rating_stars_n'].value_counts()

#### Colonna recent_purchases

In [ ]:
dfamazoncopy['recent_purchases'].value_counts()
dfamazoncopy['recent_purchases_n'] = (
    dfamazoncopy['recent_purchases']
    .astype(str)
    .str.replace('+ bought','', regex=False) # '1K+ bought ' -> '1K '
    .str.strip() #'1K ' -> '1K'
    .str.replace('K','000', regex=False) # '1K' -> '1000'
)
#TRASFORMA STRINGA IN NUMERO
dfamazoncopy['recent_purchases_n'] = pd.to_numeric(
    dfamazoncopy['recent_purchases_n'],
    errors='coerce' # trasforma tutti i valori che non possono essere convertiti in numero e i vuoti-> NaN   
).astype('Int64') # int con supporto a NaN
dfamazoncopy.drop('recent_purchases',axis=1,inplace=True)
dfamazoncopy['recent_purchases_n'].value_counts()

In [ ]:
dfamazoncopy['rating_distribution1star_%'] = (
    dfamazoncopy['rating_distribution1star']
    .astype(str)
    .str.replace('%','', regex=False) # '1K+ bought ' -> '1K '
    .str.strip() #'1K ' -> '1K'
)
#TRASFORMA STRINGA IN NUMERO
dfamazoncopy['rating_distribution1star_%'] = pd.to_numeric(
    dfamazoncopy['rating_distribution1star_%'],
    errors='coerce' # trasforma tutti i valori che non possono essere convertiti in numero e i vuoti-> NaN   
).astype('Float64') # float con supporto a NaN
print(dfamazoncopy['rating_distribution1star_%'].value_counts())
#CONVERTO NUMERO
dfamazoncopy['rating_distribution1star_%']=(
    dfamazoncopy['rating_distribution1star_%']
    .fillna(0) #tutti i NaN -> 0
    .div(100) #divide tutti i numeri / 100
)
dfamazoncopy.drop('rating_distribution1star',axis=1,inplace=True)

In [ ]:
dfamazoncopy['rating_distribution2star_%'] = (
    dfamazoncopy['rating_distribution2star']
    .astype(str)
    .str.replace('%','', regex=False) # '1K+ bought ' -> '1K '
    .str.strip() #'1K ' -> '1K'
)
#TRASFORMA STRINGA IN NUMERO
dfamazoncopy['rating_distribution2star_%'] = pd.to_numeric(
    dfamazoncopy['rating_distribution2star_%'],
    errors='coerce' # trasforma tutti i valori che non possono essere convertiti in numero e i vuoti-> NaN   
).astype('Float64') # float con supporto a NaN
#CONVERTO NUMERO
dfamazoncopy['rating_distribution2star_%']=(
    dfamazoncopy['rating_distribution2star_%']
    .fillna(0) #tutti i NaN -> 0
    .div(100) #divide tutti i numeri / 100
)
dfamazoncopy.drop('rating_distribution2star',axis=1,inplace=True)

In [ ]:
dfamazoncopy['rating_distribution3star_%'] = (
    dfamazoncopy['rating_distribution3star']
    .astype(str)
    .str.replace('%','', regex=False) # '1K+ bought ' -> '1K '
    .str.strip() #'1K ' -> '1K'
)
#TRASFORMA STRINGA IN NUMERO
dfamazoncopy['rating_distribution3star_%'] = pd.to_numeric(
    dfamazoncopy['rating_distribution3star_%'],
    errors='coerce' # trasforma tutti i valori che non possono essere convertiti in numero e i vuoti-> NaN   
).astype('Float64') # float con supporto a NaN
#CONVERTO NUMERO
dfamazoncopy['rating_distribution3star_%']=(
    dfamazoncopy['rating_distribution3star_%']
    .fillna(0) #tutti i NaN -> 0
    .div(100) #divide tutti i numeri / 100
)
dfamazoncopy.drop('rating_distribution3star',axis=1,inplace=True)

In [ ]:
dfamazoncopy['rating_distribution4star_%'] = (
    dfamazoncopy['rating_distribution4star']
    .astype(str)
    .str.replace('%','', regex=False) # '1K+ bought ' -> '1K '
    .str.strip() #'1K ' -> '1K'
)
#TRASFORMA STRINGA IN NUMERO
dfamazoncopy['rating_distribution4star_%'] = pd.to_numeric(
    dfamazoncopy['rating_distribution4star_%'],
    errors='coerce' # trasforma tutti i valori che non possono essere convertiti in numero e i vuoti-> NaN   
).astype('Float64') # float con supporto a NaN
#CONVERTO NUMERO
dfamazoncopy['rating_distribution4star_%']=(
    dfamazoncopy['rating_distribution4star_%']
    .fillna(0) #tutti i NaN -> 0
    .div(100) #divide tutti i numeri / 100
)
dfamazoncopy.drop('rating_distribution4star',axis=1,inplace=True)

In [ ]:
dfamazoncopy['rating_distribution5star_%'] = (
    dfamazoncopy['rating_distribution5star']
    .astype(str)
    .str.replace('%','', regex=False) # '1K+ bought ' -> '1K '
    .str.strip() #'1K ' -> '1K'
)
#TRASFORMA STRINGA IN NUMERO
dfamazoncopy['rating_distribution5star_%'] = pd.to_numeric(
    dfamazoncopy['rating_distribution5star_%'],
    errors='coerce' # trasforma tutti i valori che non possono essere convertiti in numero e i vuoti-> NaN   
).astype('Float64') # float con supporto a NaN
#CONVERTO NUMERO
dfamazoncopy['rating_distribution5star_%']=(
    dfamazoncopy['rating_distribution5star_%']
    .fillna(0) #tutti i NaN -> 0
    .div(100) #divide tutti i numeri / 100
)
dfamazoncopy.drop('rating_distribution5star',axis=1,inplace=True)

In [ ]:
dfamazoncopy.head(20)

### Tabelle di Aggregazione e Group By

#### Tabella di aggregazione per Recensioni per prodotto
Andando a mostrare i top 10 prodotti che hanno avuto più recensioni

<i>Ragionamento</i>: Raggruppo per <b>title</b> così da contare le recensioni e calcolare per ogni prodotto anche il rating media.
Utilizzo la colonna <b>review_id</b> come proxy per il numero di recensioni (unico per review)

In [ ]:
#tabella
product_reviews = dfamazoncopy.groupby('title').agg({
    'review_id':'count', #NUMERO DI RECENSIONI
    'rating':'mean', # RATING MEDIO 
    'helpful_vote_count':'mean', #VOTO MEDIO
}).round(2)
#Rinomino colonne
product_reviews.columns=['N_Review', 'Mean_Rating','Mean_Helpful']
#ORDINO
product_reviews.sort_values('N_Review',ascending=False).head(10)

#### Tabella di aggregazione per il Voto medio per categoria


In [ ]:
cat_rating=dfamazoncopy.groupby(['category1','category2','category3'])['rating'].agg(['mean','count']).round(2)
cat_rating.columns = ['Rating_Mean','N_reviews']
cat_rating.sort_values('N_reviews',ascending=False)

#### Pivot: Rating per brand-categoria

In [ ]:
pivot_rating = dfamazoncopy.pivot_table(
    values='rating',
    index=['category1','category2'],
    columns='category3',
    aggfunc=['mean','count'],
    fill_value=0
).round(2)
pivot_rating.head(10)

### Brand più recensiti e con sentimento positivo (per cui brand dove conviene investire)

- Anna Rudych: Il grafico 1.1 sopra rappresenta sei principali brand presenti nel dataset che hanno ricevuto il maggior volume di recensioni con un punteggio di sentiment score positivo (definito come uno score superiore a 0.3). La visualizzazione illustra i marchi che hanno generato la maggiore soddisfazione tra i clienti. 
Sull asse Y sono elencati i brand, sull asse X il numero delle recensioni positive per ciascuno. Il grafico 1.2 dimostra la distribuzione percentuale tra i top 8 brand con recensioni positive. Il brand con più recensioni positivi è "Hanes Store".  Ha il numero più elevato delle recensioni positive pari a 146. Il brand "Under Armour Store" e "Amazon Essentials Store" completano il tris dei leader con il punteggio 107 e 97. Il volume elevato delle recensioni positive è un forte indicatore di popolarità del prodotto e affitabilità del mercato. I marchi elencati hanno generato il maggior feedback entisiasta da parte dei clienti.

In [ ]:
brand_positive = (dfamazoncopy[dfamazoncopy['sentiment_score']>0.3]
                  .groupby('brand_name')
                  .size()
                  .sort_values(ascending=False)
                  .head(10)
)
#SUBPLOT 1 RIGA x 2 COLONNE
fig, (ax1, ax2) = plt.subplots(1,2,figsize=(12,8))
#Grafico 1
brand_positive.plot(
    ax=ax1,
    kind='barh',
    color='green',
    grid=True
)
ax1.set_title('Brand con maggior numero di recensioni con sentiment positivo > 0.3')
ax1.set_ylabel('Brand')
ax1.set_xlabel('Numero Recensioni')
for i, v in enumerate(brand_positive.values):
    ax1.text(v + 2, i, f'{v:,}',
            va='center',
            fontweight='bold', 
            fontsize=11,
            )

#GRAFICO 2
brand_positive.head(8).plot(
    kind='pie',
    ax=ax2
)
ax2.set_title('Top 8 Brand Positivi')

plt.show()

### Brand più recensiti e con sentimento negativo (per cui brand dove NON conviene investire)

- Anna Rudych: Il grafico 2.1 sopra rappresenta sei principali brand presenti nel dataset che hanno ricevuto il maggior volume di recensioni con un punteggio di sentiment score negativo (definito come uno score inferiore a 0.0). La visualizzazione illustra i marchi che hanno generato la minore soddisfazione tra i clienti. 
Sull asse Y sono elencati i brand, sull asse X il numero delle recensioni negative per ciascuno. Il grafico 2.2 dimostra la distribuzione percentuale tra i top 8 brand con le recensioni negative. Il brand con più recensioni negative è "Hanes Store".  Ha il numero più elevato delle recensioni negative pari a 15. Il brand "COOFANDY Store" e "POLO RALF LAUREN Store" completano il tris dei "leader" con il punteggio 9 e 8. Il volume elevato delle recensioni negative è un forte indicatore di insoddisfazione o problemi tra i clieti. I marchi elencati hanno generato il maggior feedback negativo da parte dei clienti.

In [ ]:
brand_positive = (dfamazoncopy[dfamazoncopy['sentiment_score']<0]
                  .groupby('brand_name')
                  .size()
                  .sort_values(ascending=False)
                  .head(10)
)
#SUBPLOT 1 RIGA x 2 COLONNE
fig, (ax1, ax2) = plt.subplots(1,2,figsize=(12,8))
#Grafico 1
brand_positive.plot(
    ax=ax1,
    kind='barh',
    color='red',
    grid=True
)
ax1.set_title('Brand con maggior numero di recensioni con sentiment negativo < 0')
ax1.set_ylabel('Brand')
ax1.set_xlabel('Numero Recensioni')
for i, v in enumerate(brand_positive.values):
    ax1.text(v + 2, i, f'{v:,}',
            va='center',
            fontweight='bold', 
            fontsize=11,
            )

#GRAFICO 2
brand_positive.head(8).plot(
    kind='pie',
    ax=ax2
)
ax2.set_title('Top 8 Brand Negativi')

In [ ]:
brand_positive

- Anna Rudych: Un brand ideale per un investimento dovrebbe avere un alto volume di vendite e una solida base di recensioni positive, con minimi feedback negativi, indicando un prodotto affidabile e un mercato stabile.  Il brand "Hanes Stone" ha ricevuto più recensioni sia positive che negative. Questo può essere dovuto al suo leadership sul mercato e la quantità generale delle recensioni. Invece il brand "Under Armour Store" ha preso secondo posto nel TOP con il numero delle recensioni positive pari a 107 e nello stesso momento non è stato elencato tra i brand con il punteggio maggiore delle recensioni negative. Sulla base di questi insight, si raccomanda di investire nel brand "Under Armour Store". La sua capacità di generare un volume elevato di feedback positivi, combinata con un'assenza quasi totale di recensioni con score inferiore a 0, indica un marchio maturo, affidabile e con un'eccellente percezione della qualità.

### Categorie prodotti più venduti e totale degli incassi in base alla categoria

- Anna Rudych: La categoria più venduta è Clothing, Shoes & Jewelry (Vestiti, Scarpe, Gioielleria), sono 212857 unità totale vendute e 6276 recensioni ricevute. Questo indica una domanda di mercato molto elevata. I dati evidenziano un dominio schiacciante della categoria 'Clothing Man'. Pertanto, la raccomandazione di investimento si concentra sulla categoria 'Clothing Man' nel suo insieme, mirando ai brand specifici all'interno di essa che mantengono un sentiment positivo e volumi di vendita elevati

In [ ]:
#CALCOLO RECENSIONI E INCASSI TOTALI PER CATEGORIA
cat1_sales=(dfamazoncopy[dfamazoncopy['category1'].notna() & (dfamazoncopy['category1'].str.strip()!='')]
            .groupby('category1')['price_value']
            .agg(['count','sum'])
            .reset_index()
            .rename(columns={
                'count':'n_review',
                'sum':'sum_sales'
            })
            )
ax1=(cat1_sales.set_index('category1')['sum_sales']
    .plot(
        kind='barh',
        color='steelblue',
        figsize=(12,6)
    ))
for i,v in enumerate(cat1_sales['sum_sales']):
    plt.text (v,i, f'{v:,.0f}', va='center', ha='left',fontsize='10')
    ax1.set_title('Categorie più vendute',fontsize=16,fontweight='bold')
    ax1.set_xlabel('Sum Sales')
    ax1.set_ylabel('Cateogory')
plt.show()


In [ ]:
#CALCOLO RECENSIONI E INCASSI TOTALI PER CATEGORIA
cat2_sales=(dfamazoncopy[dfamazoncopy['category2'].notna() & (dfamazoncopy['category2'].str.strip()!='')]
            .groupby('category2')['price_value']
            .agg(['count','sum'])
            .reset_index()
            .rename(columns={
                'count':'n_review',
                'sum':'sum_sales'
            })
            )
plot_cate=cat2_sales.set_index('category2')[['n_review','sum_sales']]
fig,(ax2,ax3)=plt.subplots(1,2,figsize=(25,8))
#---PRIMO SUBPLOT
ax2=plot_cate.plot(
    ax=ax2,
    kind='barh',
    figsize=(25,10)
)
for i, (val_review, val_sales) in enumerate(zip(plot_cate['n_review'],plot_cate['sum_sales'])):
    ax2.text(val_review, i-0.15, f'Reviews: {val_review}', va='center', ha='left',fontsize=9)
    ax2.text(val_sales, i+0.15, f'Sales: {val_sales}', va='center', ha='left',fontsize=9)
ax2.set_title('Categorie più vendute',fontsize=16,fontweight='bold')
ax2.set_xlabel('Review/Sales')
ax2.set_ylabel('Cateogory')
#---SECONDO SUBPLOT
plot_cate['sum_sales'].plot(
    ax=ax3,
    kind='pie',
    labels=plot_cate.index,
    autopct='%1.1f%%',
    startangle=90,
    ylabel=''
)
#LEGGENDA
ax3.legend(
    plot_cate.index,
    title='Category',
    loc='best',
    bbox_to_anchor=(1.05,0.5) #OPZIONE PER SPOSTARE FUORI
)
ax3.set_title('Vendite per categoria')
plt.tight_layout() #per non sovrapporre grafici
plt.savefig('grafico.png', dpi=300, bbox_inches='tight')
plt.show()

### Categorie prodotti più recensiti e con sentimento positivo (per cui categorie su cui investire)


In [ ]:
#COPIO DATAFRAME PER COLONNE CHE MI SERVONO
mask=dfamazoncopy['category2'].notna() & (dfamazoncopy['category2'].str.strip()!='') & (dfamazoncopy['sentiment_score']>0.3)
cat12_pose = dfamazoncopy.loc[mask,['category1','category2']].copy()
#contare recensioni
cat2_positive=(
    cat12_pose
    .groupby('category2')
    .size()
    .sort_values(ascending=False)
)
cat2_positive=cat2_positive.rename('n_positive').reset_index()
cat1_cat2_grouped = (cat12_pose
                    .groupby('category2')['category1']
                    .first()
                    .reset_index(name='category1')
)
#merge tabelle
tab_final=cat1_cat2_grouped.merge(cat2_positive,on='category2',how='right')
display(tab_final)


### Categorie prodotti più recensiti e con sentimento negativo (per cui categorie su cui NON investire)

In [ ]:
#COPIO DATAFRAME PER COLONNE CHE MI SERVONO
mask=dfamazoncopy['category2'].notna() & (dfamazoncopy['category2'].str.strip()!='') & (dfamazoncopy['sentiment_score']<0.0)
cat12_nega = dfamazoncopy.loc[mask,['category1','category2']].copy()
#contare recensioni
cat2_nega=(
    cat12_nega
    .groupby('category2')
    .size()
    .sort_values(ascending=False)
)
cat2_nega=cat2_nega.rename('n_nega').reset_index()
cat1_cat2_grouped_nega = (cat12_nega
                    .groupby('category2')['category1']
                    .first()
                    .reset_index(name='category1')
)
#merge tabelle
tab_final1=cat1_cat2_grouped_nega.merge(cat2_nega,on='category2',how='right')
display(tab_final1)


In [ ]:
#COPIO DATAFRAME PER COLONNE CHE MI SERVONO
mask=dfamazoncopy['category2'].notna() & (dfamazoncopy['category2'].str.strip()!='') & (dfamazoncopy['sentiment_score']>=0.0) & (dfamazoncopy['sentiment_score']<=0.3)
cat12_neutre = dfamazoncopy.loc[mask,['category1','category2']].copy()
#contare recensioni
cat2_neutre=(
    cat12_neutre
    .groupby('category2')
    .size()
    .sort_values(ascending=False)
)
cat2_neutre=cat2_neutre.rename('n_neutre').reset_index()
cat1_cat2_grouped = (cat12_neutre
                    .groupby('category2')['category1']
                    .first()
                    .reset_index(name='category1')
)
#merge tabelle
tab_final2=cat1_cat2_grouped.merge(cat2_neutre,on='category2',how='right')
tab_final2=tab_final2.merge(cat2_nega,on='category2',how='left')
tab_final2=tab_final2.merge(cat2_positive,on='category2',how='left')

# Riempio eventuali NaN con 0 per evitare errori nel calcolo
tab_final2[['n_positive', 'n_neutre', 'n_nega']] = tab_final2[['n_positive', 'n_neutre', 'n_nega']].fillna(0)

# Creo la nuova colonna calcolando la percentuale
tab_final2['% positive'] = (
    tab_final2['n_positive'] / (tab_final2['n_positive'] + tab_final2['n_neutre'] + tab_final2['n_nega']) * 100
).round(2)
tab_final2['% neutre'] = (
    tab_final2['n_neutre'] / (tab_final2['n_positive'] + tab_final2['n_neutre'] + tab_final2['n_nega']) * 100
).round(2)
tab_final2['% negative'] = (
    tab_final2['n_nega'] / (tab_final2['n_positive'] + tab_final2['n_neutre'] + tab_final2['n_nega']) * 100
).round(2)
display(tab_final2)

fig, ax2 = plt.subplots(figsize=(25, 10))

# solo le 3 colonne di conteggio nel barh
tab_final2[['n_neutre','n_nega','n_positive']].plot(
    kind='barh',
    ax=ax2,
    color=['orange', 'red', 'green'],
    alpha=0.8
)

# etichette con valore + percentuale
for i, (idx, row) in enumerate(tab_final2.iterrows()):
    vn, vneg, vpos = row['n_neutre'], row['n_nega'], row['n_positive']
    pn, pneg, ppos = row['% neutre'], row['% negative'], row['% positive']

    ax2.text(vn  + 5, i - 0.25, f'Neu: {int(vn)} ({pn}%)',   va='center', ha='left', fontsize=9)
    ax2.text(vneg+ 5, i,        f'Neg: {int(vneg)} ({pneg}%)', va='center', ha='left', fontsize=9, color='red')
    ax2.text(vpos+ 5, i + 0.25, f'Pos: {int(vpos)} ({ppos}%)', va='center', ha='left', fontsize=9, color='green')

ax2.set_title('Categorie più recensite per sentiment', fontsize=16, fontweight='bold')
ax2.set_xlabel('Numero recensioni')
ax2.set_ylabel('Category2')
ax2.legend(title='Sentiment', loc='upper right')

plt.tight_layout()
plt.savefig('grafico_categorie_sentiment.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
tab_plot = tab_final2.copy()
tab_plot['tot']=tab_plot['n_positive']+tab_plot['n_neutre']+tab_plot['n_nega']
tab_plot = tab_plot.sort_values('tot',ascending=False).head(9)

n_cat=len(tab_plot)
n_cols = 3
nrows = math.ceil(n_cat/n_cols)

fig,axes=plt.subplots(nrows,n_cols, figsize=(5*n_cols,5*nrows))
#SE C'E' SOLO UNA RGA/COLONNA AXES E' 2D AL CONTRARIO FORZO AD 1D
axes = axes.flatten() if isinstance(axes, (list,np.ndarray)) else [axes] 
labels=['Positive', 'Neutre','Negative']
colors=['green','orange','red']
for i, (idx,row) in enumerate(tab_plot.iterrows()):
    ax=axes[i]
    sizes=[row['% positive'], row['% neutre'], row['% negative']]
  

    wedges, texts, autotexts = ax.pie(
        sizes,
        labels=None,
        colors=colors,
        autopct='%1.1f%%',
        startangle=180,
        pctdistance=0.5
    )
    ax.set_title(row['category2'], fontsize=11, fontweight='bold')
    ax.axis('equal') #fai per tutti un cerchio

#Legenda Unica
fig.legend(
    wedges,
    labels,
    loc='upper right',
    bbox_to_anchor=(1.05,1),
    title='Sentiments'
)
#Titolo Generale
plt.suptitle('Distribuzione % Sentiment per Category', fontsize=14, fontweight='bold', y=0.98)
plt.tight_layout()
plt.subplots_adjust(top=0.90, right=0.85) # SPAZIO BIANCO VERSO TITOLO E LEGENDA
plt.show()

### Prodotti più recensiti e con sentimento positivi

In [ ]:
#RECENSIONI POSITIVE
mask=dfamazoncopy['title'].notna() & (dfamazoncopy['title'].str.strip()!='') & (dfamazoncopy['sentiment_score']>0.3)
product_positive = dfamazoncopy.loc[mask,['title','brand_name','sentiment_score']].copy()
#contare recensioni
product_reviews_positive=(
    product_positive
    .groupby('title')['sentiment_score']
    .count()
    .sort_values(ascending=False)
    .rename('n_reviews_positive')
    .reset_index()
)

#RECENSIONI NEUTRE
mask=dfamazoncopy['title'].notna() & (dfamazoncopy['title'].str.strip()!='') & (dfamazoncopy['sentiment_score']>=0.0) & (dfamazoncopy['sentiment_score']<=0.3)
product_neutre = dfamazoncopy.loc[mask,['title','brand_name','sentiment_score']].copy()
#contare recensioni
product_reviews_neutre=(
    product_neutre
    .groupby('title')['sentiment_score']
    .count()
    .sort_values(ascending=False)
    .rename('n_reviews_neutre')
    .reset_index()
)

#RECENSIONI NEGATIVE
mask=dfamazoncopy['title'].notna() & (dfamazoncopy['title'].str.strip()!='') & (dfamazoncopy['sentiment_score']<0.0)
product_negative = dfamazoncopy.loc[mask,['title','brand_name','sentiment_score']].copy()
#contare recensioni
product_reviews_negative=(
    product_negative
    .groupby('title')['sentiment_score']
    .count()
    .sort_values(ascending=False)
    .rename('n_reviews_negative')
    .reset_index()
)

#MERGE
tab_final_p_r=product_reviews_positive.merge(product_reviews_neutre,on='title',how='right')
tab_final_p_r=tab_final_p_r.merge(product_reviews_negative,on='title',how='right')

#RIEMPIO tutti i NAN con 0
tab_final_p_r=tab_final_p_r.fillna(0)

#ORDINO TABELLA
tab_final_p_r['n_reviews_tot']=(
    tab_final_p_r['n_reviews_positive']+
    tab_final_p_r ['n_reviews_negative']+
    tab_final_p_r['n_reviews_neutre']
)

tab_final_p_r=tab_final_p_r.sort_values(
    by='n_reviews_tot',
    ascending=False
).reset_index(drop=True)

display(tab_final_p_r)

In [ ]:
# prendo solo le prime 10 righe
topp10=tab_final_p_r.head(10).copy()

#accorcio stringa titolo
topp10['title_short']=topp10['title'].str.slice(0,50)+'...'

#creo indice
topp10_plot=topp10.set_index('title_short')[['n_reviews_positive','n_reviews_neutre','n_reviews_negative']]

#PLOT KIND BARH
ax=topp10_plot.plot(
    kind='barh',
    figsize=(16,8),
    color=['green','orange','red'],
    alpha=0.8
)
ax.set_title('Top 10 prodotti per numero di recensioni per sentiment', fontsize=16, fontweight='bold')
ax.set_xlabel('N. Reviews')
ax.set_ylabel('Prodotto')
ax.legend(['Positive','Neutre','Negative'],title='Sentiment',loc='upper right')

#ETICHETTE
for container in ax.containers: # un container per ogni serie pos-net-neg
    ax.bar_label(container, fmt='%d', padding=3, fontsize=8)

plt.tight_layout()
plt.show()


In [ ]:
for _,row in topp10.iterrows():
    titolo=row['title_short']
    valori=row[
       ['n_reviews_positive',
        'n_reviews_neutre',
        'n_reviews_negative']].copy()
    valori.index=labels
    labels=['positivo','neutro','negativo']
   
    #prodotto con 0 recensioni
    if sum(valori)==0:
        continue #salto

    plt.figure(figsize=(5,5))
    ax=valori.plot(
        kind='pie',
        autopct='%1.1f%%',
        startangle=90,
        ylabel=''
    )
    plt.title(titolo)
    plt.tight_layout()
    plt.show()


In [ ]:
cols=['n_reviews_positive','n_reviews_neutre','n_reviews_negative']
labels = ['positivo', 'neutro', 'negativo']
n= len(topp10)
ncols=3
nrows=math.ceil(n/ncols)

fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(5*ncols,5*nrows))

#axes come array 2Dimensioni anche se ha una sola riga
axes=axes.reshape(nrows,ncols)

for idx, (_,row) in enumerate (topp10.iterrows()):
    r = idx//ncols
    c = idx%ncols
    ax = axes[r,c]

    if int(row['n_reviews_tot']) == 0:
        ax.axis('off')
        continue
    
    s = row[cols].copy()
    s.index = labels

    s.plot(
        kind='pie',
        ax=ax,
        autopct='%1.1f%%',
        startangle=90,
        ylabel=''
    )
    ax.set_title(row['title_short'])

#assi vuoti se n non è multiplo di 3
for j in range(idx+1, nrows*ncols):
    r= j // ncols
    c= int(j) % ncols
    axes[r,c].axis('off')


plt.tight_layout()
plt.show()
